In [1]:
import os

In [2]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

# Clone Github repositories


In [3]:
%pwd

'/Users/hackthebox/Downloads/Source-Code-Analysis-using-GenAI/research'

In [4]:
!mkdir test_repo

In [5]:
repo_path = "test_repo/"

Repo.clone_from("https://github.com/entbappy/End-to-end-ML-Project-Implementation", to_path=repo_path)

<git.repo.base.Repo '/Users/hackthebox/Downloads/Source-Code-Analysis-using-GenAI/research/test_repo/.git'>

In [6]:
repo_path = "test_repo/"

loader = GenericLoader.from_filesystem(repo_path+'/src/mlProject',
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [7]:
documents = loader.load()

In [8]:
documents

[Document(page_content='import os\nimport sys\nimport logging\n\nlogging_str = "[%(asctime)s: %(levelname)s: %(module)s: %(message)s]"\n\nlog_dir = "logs"\nlog_filepath = os.path.join(log_dir,"running_logs.log")\nos.makedirs(log_dir, exist_ok=True)\n\n\nlogging.basicConfig(\n    level= logging.INFO,\n    format= logging_str,\n\n    handlers=[\n        logging.FileHandler(log_filepath),\n        logging.StreamHandler(sys.stdout)\n    ]\n)\n\nlogger = logging.getLogger("mlProjectLogger")', metadata={'source': 'test_repo/src/mlProject/__init__.py', 'language': <Language.PYTHON: 'python'>}),
 Document(page_content='from mlProject.config.configuration import ConfigurationManager\nfrom mlProject.components.model_trainer import ModelTrainer\nfrom mlProject import logger\n\n\n\nSTAGE_NAME = "Model Trainer stage"\n\nclass ModelTrainerTrainingPipeline:\n    def __init__(self):\n        pass\n\n    def main(self):\n        config = ConfigurationManager()\n        model_trainer_config = config.get

# Chunkings

In [9]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 2000,
                                                             chunk_overlap = 200)

In [10]:
texts = documents_splitter.split_documents(documents)

In [11]:
len(texts)

19

# Embedding model

In [12]:
os.environ["OPENAI_API_KEY"] = "*********************************"

In [13]:
embeddings=OpenAIEmbeddings(disallowed_special=())

# Knowledge base (vector DB)

In [14]:
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./data')
vectordb.persist()

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors..
Retrying langchain.embeddings.openai.embed_wit

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

# LLM Wrapper


In [16]:
# llm = ChatOpenAI(model_name="gpt-4")
llm = ChatOpenAI()

In [17]:
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

In [18]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":3}), memory=memory)

NameError: name 'vectordb' is not defined

# Q&A

In [19]:
question = "what is DataIngestion class?"

In [20]:
result = qa(question)
print(result['answer'])

NameError: name 'qa' is not defined